In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual, BoundedIntText, FloatLogSlider
from IPython.display import display
sns.set()

Set up number of points to use on plots

In [3]:
NumPoints = 200

Set up conversions from natural units to SI units

In [4]:
# 4D reduced Planck mass measured in GeV
M4 = 2.435E18

# in units where c = hbar = kBoltzmann = 1, the Planck temperature is
# TP = 1/sqrt(G) = 1.41678416E32 Kelvin
#    = sqrt(8pi) / sqrt(8piG) = sqrt(8pi) * M4
# that gives us 1 Kelvin measured in GeV
Kelvin = np.sqrt(8*np.pi) * M4 / 1.41678416E32

# in units where c = hbar = 1, the Planck length is
# ellP = sqrt(G) = 1.61625518E-35 m
#      = sqrt(8piG) / sqrt(8pi) = 1 / (M4 * sqrt(8pi))
# that gives us 1 metre measured in 1/GeV
Metre = 1.0 / (M4 * np.sqrt(8*np.pi) * 1.61625518E-35)
Kilometre = 1000.0 * Metre
Mpc = 3.08567758128E+19 * Kilometre

# in units where c = hbar = 1, the Planck mass is
# MP = 1/sqrt(G) = 2.17643424E-8 kg
#    = sqrt(8pi) / sqrt(8piG) = sqrt(8pi) * M4
# that gives us 1 kg measured in GeV
Kilogram = np.sqrt(8*np.pi) * M4 / 2.17643424E-8
Gram = Kilogram/1000.0

The *ModelParameters* class captures details of the 4D and 5D Planck masses, and uses these to compute derived quantities such as the brane tension and the crossover temperature (in GeV and Kelvin) from the quadratic Hubble regime to the linear regime. It also computes the AdS radius in inverse GeV and metres, assuming that the bulk cosmological constant is tuned to produce zero four-dimensional cosmological constant. This is important for deciding when the transition takes place from a "small" 5D black hole (approximated by Myers-Perry) to a "large", effectively 4D black hole (approximated by 4D Schwarzschild).

In [5]:
class ModelParameters:

    def __init__(self, M5):
        # M5 is the fundamental 5D *reduced* Planck mass, measured in GeV
        # M4 is the derived 4D *reduced* Planck mass, measured in GeV

        # M5 should not be much less than a TeV
        if M5 < 1E3:
            raise RuntimeError('Parameter error: 5D Planck mass M5 is less than 1 TeV')

        if M4 <= M5:
            raise RuntimeError('Parameter error: 4D Planck mass should be more (usually much more) than the 5D Planck mass')

        # store primary data
        self.M5 = M5
        self.M4 = M4

        # M_ratio < 1 (usually mu << 1) is the M5/M4 ratio
        M_ratio = M5/M4
        self.M_ratio = M_ratio

        # compute brane tension lambda = 6 mu^2 M5^4
        # note we can't actually call it lambda in Python, for which lambda is a reserved word
        self.tension = 6 * M_ratio*M_ratio * M5*M5*M5*M5

        # also compute the mass scale associated with the tension
        self.tension_scale = np.power(6*M_ratio*M_ratio, 1.0/4.0) * M5

        # compute crossover temperature from the quadratic to the linear regime, which occurs when rho = 2 lambda

        # we need the 4D radiation density constant (per degree of freedom) to convert temperature to energy density
        RadiationConstant = np.pi*np.pi / 30.0
        self.RadiationConstant = RadiationConstant

        # assume the crossover temperature is high enough that all SM particles are relativistic and in thermal
        # equilibrium, which should be good above Tcross = 200 GeV; we insist Tcross > 1E3 (see below),
        # which is hopefully enough headroom
        gstar = 106.7
        self.gstar = gstar

        self.T_crossover = np.power(12.0 / (RadiationConstant * gstar) * M_ratio*M_ratio, 1.0/4.0) * M5
        self.T_crossover_Kelvin = self.T_crossover / Kelvin

        # compute AdS radius ell and its inverse, mu
        self.mu = M_ratio*M_ratio * M5
        self.ell_AdS = 1.0/self.mu
        self.ell_AdS_Metres = self.ell_AdS / Metre

        # SANITY CHECKS

        # check that brane tension scale is smaller than 5D Planck scale
        # this is basically implied if M4 > M5, but there are numerical factors that make it
        # worth checking
        if self.tension_scale > M5:
            raise RuntimeError('Parameter error: brane tension lambda = {tension:.3g} GeV should be smaller than '
                               '5D Planck scale {M5:.3g} GeV'.format(tension=self.tension, M5=M5))

        # check that crossover temperature is smaller than 5D Planck scale
        # this is basically implied if M4 > M5, but there are numerical factors that make it
        # worth checking
        if self.T_crossover > M5:
            raise RuntimeError('Parameter error: crossover temperature T_cross = {Tcross:.3g} GeV should be smaller than '
                               '5D Planck scale {M5:.3g} GeV'.format(Tcross=self.T_crossover, M5=M5))

        # there's no need to compare the tension and crossover temperature to the 4D Planck scale, because
        # we've already guaranteed that the 4D Planck scale is larger than the 5D Planck scale

        # check that crossover temperature is larger than suggested experimental minimum 1E3 GeV from Guedens et al.,
        # see Eq. (18) and discussion below in astro-ph/0205149v2
        if self.T_crossover < 1E3:
            raise RuntimeError('Parameter error: crossover temperature T_cross = {Tcross:.3g} GeV should be larger than '
                               'experimental limit 1E3 GeV = TeV suggested by Guedens et al.'.format(Tcross=self.T_crossover))

    def __str__(self):
        return '5D Planck Mass        M5      = {M5:.5g} GeV\n' \
               '4D Planck Mass        M4      = {M4:.5g} GeV\n' \
               '5D/4D mass ratio      M5/M4   = {ratio:.5g}\n' \
               'brane tension         lambda  = {tension:.5g} GeV^4\n' \
               'brane tension scale           = {tscale:.5g} GeV\n' \
               'crossover temperature T_cross = {Tcross:.5g} GeV\n' \
               '                              = {TcrossKelvin:.5g} K\n' \
               'AdS curvature length  ell_AdS = {ell:.5g} / GeV\n' \
               '                              = {ellMetre:.5g} m\n' \
               'AdS curvature scale   mu      = {mu:.5g} GeV'.format(M5=self.M5, M4=self.M4, tension=self.tension,
                                                                     tscale=self.tension_scale, Tcross=self.T_crossover,
                                                                     TcrossKelvin=self.T_crossover_Kelvin,
                                                                     ell=self.ell_AdS, ellMetre=self.ell_AdS_Metres,mu=self.mu,
                                                                     ratio=self.M_ratio)

In [6]:
params = ModelParameters(5E8)
print(params)

5D Planck Mass        M5      = 5e+17 GeV
4D Planck Mass        M4      = 2.435e+18 GeV
5D/4D mass ratio      M5/M4   = 0.20534
brane tension         lambda  = 1.5812e+70 GeV^4
brane tension scale           = 3.546e+17 GeV
crossover temperature T_cross = 1.7325e+17 GeV
                              = 2.0107e+30 K
AdS curvature length  ell_AdS = 4.7434e-17 / GeV
                              = 9.3587e-33 m
AdS curvature scale   mu      = 2.1082e+16 GeV


The *CosmologyEngine* class provides methods to compute the Hubble rate, Hubble length, horizon mass, etc.

In [7]:
class CosmologyEngine:

    def __init__(self, params: ModelParameters):
        self.params = params

    # compute the radiation energy density in GeV^4 from a temperature supplied in GeV
    # currently, we assume there are a fixed number of relativistic species
    def rho_radiation(self, T):
        # check that supplied temperature is lower than the 5D Planck mass
        if T > self.params.M5:
            raise RuntimeError('Temperature T={temp:.3g} GeV is higher than the 5D Planck mass M5={M5:.3g} GeV'.format(temp=T, M5=self.params.M5))

        return params.RadiationConstant * params.gstar * T*T*T*T

    # compute the Hubble rate in GeV at a time corresponding to a temperature supplied in GeV
    def Hubble(self, T):
        rho = self.rho_radiation(T)

        return 1.0 / (np.sqrt(3) * params.M4) * np.sqrt(rho * (1.0 + rho/(2.0 * params.tension)))

    # compute the 4D-only Hubble rate in GeV at a time corresponding to a temperature supplied in GeV
    def Hubble4(self, T):
        rho = self.rho_radiation(T)

        return 1.0 / (np.sqrt(3) * params.M4) * np.sqrt(rho)

    # compute the Hubble length in 1/GeV at a time corresponding to a temperature supplied in GeV
    # the formula here is R_H = 1/H
    def R_Hubble(self, T):
        return 1.0/self.Hubble(T)

    # compute the 4D-only Hubble length in 1/GeV at a time corresponding to a temperature supplied in GeV
    def R_Hubble4(self, T):
        return 1.0/self.Hubble4(T)

    # compute the mass (in GeV) enclosed within the Hubble length, at a time corresponding to a temperature supplied in GeV
    # the formula here is M_H = (4/3) pi rho R_H^3, but we compute it directly to avoid multiple evaluations of rho
    def M_Hubble(self, T):
        rho = self.rho_radiation(T)
        M_H = 4.0 * np.sqrt(3) * np.pi * params.M4*params.M4*params.M4 * np.power(1.0 + rho/(2.0 * params.tension), -3.0/2.0) / np.sqrt(rho)

        return M_H

    # compute the mass (in GeV) enclosed within the 4D-only Hubble length, at a time corresponding to a temperature supplied in GeV
    def M_Hubble4(self, T):
        rho = self.rho_radiation(T)
        M_H = 4.0 * np.sqrt(3) * np.pi * params.M4*params.M4*params.M4 / np.sqrt(rho)

        return M_H

Generate plot of PBH mass and corresponding collapse lengthscale as a function of the initial temperature

In [8]:
def PBHMassPlot(M5, Tlo=1E3, Thi=None, units='grams'):
    # check desired units are sensible
    if units not in ['grams', 'kilograms', 'GeV']:
        units = 'grams'

    if Thi is None:
        Thi = M5

    # build a dictionary of unit conversion coefficients
    units_conversion = {'grams': Gram, 'kilograms': Kilogram, 'GeV': 1.0}

    params = ModelParameters(M5)
    engine = CosmologyEngine(params)

    T_range = np.geomspace(Tlo, Thi, num=NumPoints)

    unit = units_conversion[units]

    M_values = [engine.M_Hubble(T) / unit for T in T_range]
    M4_values = [engine.M_Hubble4(T) / unit for T in T_range]
    
    plt.figure()
    plt.loglog(T_range, M_values, label='Randall-Sundrum')
    plt.loglog(T_range, M4_values, label='Standard 4D')
    plt.xlabel('Temperature T / GeV')
    plt.ylabel('PBH mass at formation / {units}'.format(units=units))
    plt.legend()
    plt.show()

In [9]:
PBHMassPlot(5E17, units='grams')

<IPython.core.display.Javascript object>

In [18]:
engine = CosmologyEngine(params)
print(engine.M_Hubble(1E8) / Gram)
print(engine.M_Hubble4(1E8) / Gram)

In [19]:
def PBHLifetime(M5, M4, Ti, Mi):
    # M5 is the fundamental 5D *reduced* Planck mass
    # M4 is the derived 4D *reduced* Planck mass
    # Ti is the initial temperature of the radiation bath, measured  as an energy in GeV
    # Tform is the time at which the PBH forms, measured using the temperature of the radiation bath

    # compute the brane tension lambda from the 4d and 5d Planck scales
    # lambda has units of (GeV)^4
    # note we can't actually call it lambda in Python, because that's a reserved word
    M5_cube = M5*M5*M5
    M5_six = M5_cube*M5_cube
    M4_square = M4*M4
    tension = 6.0 * M5_six / M4_square

    # compute crossover temperature between quadratic and linear regimes
    # assume Tcross is high enough that all SM particles are relativistic and in thermal equilibrium,
    # which should be good above about Tcross = 200 GeV.
    gstar = 106.75
    Tcross = np.pow((60.0 / (np.pi*np.pi)) * tension / gstar, 1.0/4.0)

    if Tcross < 300.0:
        raise RuntimeError('Warning: Tcross is too low for assumptions of thermal equilibrium for all SM species')

    # Guedens et al. (astro-ph/025149) suggest that the minimum value allowed by experiment is ~ 10^3 GeV
    if Tcross < 1E3:
        raise RuntimeError('Warning: Tcross violates Guedens et al. approximate lower bound')


<IPython.core.display.Javascript object>